# DATABRICKS #3 Queries
Reminder: This notebook is to be run inside of DATABRICKS.

#### Import

In [ ]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [ ]:
%sql
SET spark.databricks.delta.formatCheck.enabled=false

# CREATE & CLEAN DATAFRAMES

## DataFrame for pinterest post data

In [ ]:
# Pinterest Post Data

# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_pin = "/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.pin/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"

# DataFrame for pinterest post data
df_pin = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_pin)


## CLEANING

null_info_dict = {  'title':            'No Title Data Available',
                    'description':      'No description available Story format',
                    'poster_name':      'User Info Error',
                    'follower_count':   'User Info Error',
                    'tag_list':         'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e',
                    'image_src':        'Image src error.',
                    }

#Setup Cleaned DataFrame variable
cleaned_dfp = df_pin

# Replace Error Values with None
cleaned_dfp = cleaned_dfp.replace({'No Title Data Available': None},                  subset=['title'])
cleaned_dfp = cleaned_dfp.replace({'No description available Story format': None},    subset=['description'])
cleaned_dfp = cleaned_dfp.replace({'User Info Error': None},                          subset=['poster_name'])
cleaned_dfp = cleaned_dfp.replace({'User Info Error': None},                          subset=['follower_count'])
cleaned_dfp = cleaned_dfp.replace({'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e': None},        subset=['tag_list'])
cleaned_dfp = cleaned_dfp.replace({'Image src error.': None},                         subset=['image_src'])


#follower_count column contains k and M letters refering to thousand and million, respectively. 
#we'll use regexp replace to correct these so we can cast column as integer.

cleaned_dfp = cleaned_dfp.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
cleaned_dfp = cleaned_dfp.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))

#change data type of numeric columns into int
cleaned_dfp = cleaned_dfp.withColumn("follower_count", cleaned_dfp["follower_count"].cast("integer"))
cleaned_dfp = cleaned_dfp.withColumn("downloaded", cleaned_dfp["downloaded"].cast("integer"))
cleaned_dfp = cleaned_dfp.withColumn("index", cleaned_dfp["index"].cast("integer"))

#clean datain save_location
cleaned_dfp = cleaned_dfp.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

#rename index column to ind
cleaned_dfp = cleaned_dfp.withColumnRenamed("index", "ind") 

#re-order the DataFrame

cleaned_dfp = cleaned_dfp.select    (   "ind",
                                        "unique_id",
                                        "title",
                                        "description",
                                        "follower_count",
                                        "poster_name",
                                        "tag_list",
                                        "is_image_or_video",
                                        "image_src",
                                        "save_location",
                                        "category"
                                        )


#Setup Cleaned DataFrame variable
df_pin = cleaned_dfp

display(df_pin)
# Print Schema 
df_pin.printSchema()


ind,unique_id,title,description,follower_count,poster_name,tag_list,is_image_or_video,image_src,save_location,category
4387,ae5e7377-f1bd-4ac5-94de-bee317f51a43,Βάπτιση: H παραμυθένια βάπτιση της Τιτίκας με θέμα το μονόκερο από την e.m. for you,Το όνομα που επέλεξε η μαμά Ανδριανή για τη γλυκιά Τιτίκα δεν είναι καθόλου τυχαίο. Και φυσικά δεν άφησε τίποτα στην τύχη ούτε την ημέρα της βάπτισης. Ανέθεσε την οργάνωση στην…,4,Manosbojana Katsareas,"Diy Flowers,Flower Diy,Baptism Decorations,Christening,Event Planning,Wedding Planner,Baptism Ideas,Birthday,Party",image,https://i.pinimg.com/originals/db/aa/d2/dbaad28fa85012a4ea6958540d98a8e5.jpg,/data/event-planning,event-planning
6717,bc5ab9ee-505e-44f6-92ba-677fe4fdf3e3,〚 Уютные шведские коттеджи от Carina Olander 〛 ◾ Фото ◾ Идеи ◾ Дизайн,"Традиционные шведские коттеджи, обычно с красным фасадом — это настоящее воплощением идеального зимнего уюта. Они обычно оформлены очень просто и ✌PUFIK. Beautiful Interiors. On…",136000,PUFIK Interiors & Inspirations,"Scandinavian Cottage,Swedish Cottage,Swedish Home Decor,Swedish Farmhouse,Swedish Style,Swedish Kitchen,Kitchen Black,Swedish House,Cozy Cottage",image,https://i.pinimg.com/originals/32/eb/72/32eb72e4fd8654c115a64528bd1f34b4.png,/data/home-decor,home-decor
6633,d136f6bc-840d-44f8-bbad-115eb7e6c51e,The Cottage Journal on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are giving. If you could paint your cabinets any…”,"6,636 Likes, 141 Comments - The Cottage Journal (@thecottagejournal) on Instagram: “Can you say color?! 😍😍😍 We are loving the cheery vibes that these aqua blue cabinets are g…",394,Sarah Martin,"Diy Kitchen Cabinets,Kitchen Redo,Home Decor Kitchen,New Kitchen,Home Kitchens,Kitchen Remodeling,Aqua Kitchen,Kitchen Counters,Kitchen Islands",image,https://i.pinimg.com/originals/8c/17/a2/8c17a257b70780480bb89c3699363144.jpg,/data/home-decor,home-decor
1706,b5c8a1b5-9e90-4522-9bec-2477b698d5b7,Standing Figurine Toys Xmas Santa Claus Snowman Reindeer Figure Plush Dolls Christmas Decorations Ornaments Home Indoor Table Ornaments Christmas Party Tree Hanging Decor Toys Gifts for Kids Friends…,"Features: Material:Lint Size:48ｘ18cm Quantity:1 pc Shape:Santa Claus, snowman. Elk Occasion:Christmas Description: 1. Fashion design, high quality 2. Santa Claus, snowman. Elk C…",5000,Wear24-7,"Merry Christmas To You,Christmas Toys,Great Christmas Gifts,Christmas Snowman,Christmas Ornaments,Holiday,Christmas Party Decorations,Christmas Themes,Decoration Party",image,https://i.pinimg.com/originals/b5/7f/21/b57f219fa89c1165b57525b8eae711da.jpg,/data/christmas,christmas
1676,3ed92c2d-9cca-4ccf-ac25-44a9d8bec919,PAUPOO™ Pre-lit Artificial Christmas Hanging Basket - Flocked with Mixed Decorations and White LED Lights - Frosted Berry BUY 5 GET 2 FREE(7PACKS),"❤️ ❤️ MERRY CHRISTMAS ❤️ ❤️ ❤️ ❤️ Early Christmas Special:Buy 3 Get 1 Free, Buy 5 Get 2 Free,Deadline November 25. Color:GreenMaterial:Polyvinyl ChlorideItem Dimensions:LxWxH 20…",784,paupoo,"Christmas Hanging Baskets,Christmas Plants,Christmas Wreaths,Christmas Ornaments,Merry Christmas,Christmas Sale,Christmas Porch Ideas,Hanging Christmas Lights,Christmas Island",image,https://i.pinimg.com/originals/ef/40/7e/ef407e9568aa46fed4162bd1fd28786e.jpg,/data/christmas,christmas
2482,08604f20-fa17-4b9a-9949-781717eca6cd,FORNT PORCH CHRISTMAS DECORATING IDEAS,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",46000,"Life on Summerhill | Home, Holiday Decor & DIY Website","Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",video,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,/data/chr

root
-- ind: integer (nullable = true)
-- unique_id: string (nullable = true)
-- title: string (nullable = true)
-- description: string (nullable = true)
-- follower_count: integer (nullable = true)
-- poster_name: string (nullable = true)
-- tag_list: string (nullable = true)
-- is_image_or_video: string (nullable = true)
-- image_src: string (nullable = true)
-- save_location: string (nullable = true)
-- category: string (nullable = true)

## DataFrame for pinterest geo data

In [ ]:
# Pinterest Geo Data

# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_geo = "/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.geo/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"

# DataFrame for pinterest geo data
df_geo = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_geo)

#Setup Cleaned DataFrame variable
cleaned_dfg = df_geo

## CLEANING

from pyspark.sql.functions import array
cleaned_dfg = cleaned_dfg.withColumn("coordinates", array("latitude", "longitude"))

cleaned_dfg = cleaned_dfg.drop("latitude", "longitude")

from pyspark.sql.functions import to_timestamp
cleaned_dfg = cleaned_dfg.withColumn("timestamp", to_timestamp("timestamp"))

cleaned_dfg = cleaned_dfg.select    (   "ind",
                                        "country",
                                        "coordinates",
                                        "timestamp"
                                        )

#Setup Cleaned DataFrame variable
df_geo = cleaned_dfg

display(df_geo)
# Print Schema 
df_geo.printSchema()


ind,country,coordinates,timestamp
9455,British Indian Ocean Territory (Chagos Archipelago),"List(-82.9272, -150.346)",2022-03-15T01:46:32.000+0000
6814,British Indian Ocean Territory (Chagos Archipelago),"List(-86.5675, -149.565)",2022-09-02T11:34:28.000+0000
5111,British Indian Ocean Territory (Chagos Archipelago),"List(-83.7472, 8.65953)",2021-04-01T00:56:57.000+0000
10073,Antarctica (the territory South of 60 deg S),"List(-32.8885, -170.295)",2021-06-29T19:56:04.000+0000
10073,Antarctica (the territory South of 60 deg S),"List(-32.8885, -170.295)",2021-06-29T19:56:04.000+0000
2418,Antarctica (the territory South of 60 deg S),"List(-88.4642, -171.061)",2022-05-27T11:30:59.000+0000
2418,Antarctica (the territory South of 60 deg S),"List(-88.4642, -171.061)",2022-05-27T11:30:59.000+0000
5162,Antarctica (the territory South of 60 deg S),"List(-71.6607, -149.206)",2019-09-27T19:06:43.000+0000
1335,Antarctica (the territory South of 60 deg S),"List(-77.9931, -175.682)",2022-03-19T17:29:42.000+0000
9185,Antarctica (the territory South of 60 deg S),"List(-10.3764, -22.9809)",2019-10-06T18:12:55.000+0000


root
-- ind: long (nullable = true)
-- country: string (nullable = true)
-- coordinates: array (nullable = false)
 |-- element: double (containsNull = true)
-- timestamp: timestamp (nullable = true)

## DataFrame for pinterest user data

In [ ]:
# Pinterest User Data

# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
file_location_user = "/mnt/user-0e2a0bfcc015-bucket/topics/0e2a0bfcc015.user/partition=0/*.json"
file_type = "json"
# Ask Spark to infer the schema
infer_schema = "true"

# DataFrame for pinterest user data
df_user = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location_user)


## CLEANING

#Setup Cleaned DataFrame variable
cleaned_dfu = df_user

# Concatenate the first_name and last_name columns to create the user_name column
cleaned_dfu = cleaned_dfu.withColumn("user_name", concat("first_name", "last_name"))

cleaned_dfu = cleaned_dfu.drop("first_name", "last_name")

from pyspark.sql.functions import to_timestamp
cleaned_dfu = cleaned_dfu.withColumn("date_joined", to_timestamp("date_joined"))

cleaned_dfu = cleaned_dfu.select    (   "ind",
                                        "user_name",
                                        "age",
                                        "date_joined"
                                        )

#Setup Cleaned DataFrame variable
df_user = cleaned_dfu

display(df_user)
# Print Schema 
df_user.printSchema()

ind,user_name,age,date_joined
6353,ChristopherHernandez,42,2017-02-18T00:31:22.000+0000
2015,ChristopherBradshaw,27,2016-03-08T13:38:37.000+0000
10673,AlexanderCervantes,59,2017-05-12T21:22:17.000+0000
1857,ChristopherHamilton,48,2016-02-27T16:57:44.000+0000
10020,ChristopherHawkins,45,2016-09-15T06:02:53.000+0000
2041,ChristopherCampbell,35,2015-10-22T22:42:23.000+0000
7031,ChristopherAnderson,48,2016-06-13T17:09:14.000+0000
6398,ChristinaDavenport,39,2016-06-29T20:43:59.000+0000
3599,AlexandriaAlvarado,20,2015-10-23T04:13:23.000+0000
4256,AlexandriaAlvarado,20,2015-10-23T04:13:23.000+0000


root
-- ind: long (nullable = true)
-- user_name: string (nullable = true)
-- age: long (nullable = true)
-- date_joined: timestamp (nullable = true)

# DataFrame Queries
The following queries are for the aiCore Pinterest Data Pipeling Project - mileston 7: Batch Processing

## 1 - Find which was the most popular category in each country.

In [ ]:
joined_pin_geo = df_pin.join(df_geo, df_pin["ind"] == df_geo["ind"], how="inner")

# Creating a Window specification
window_spec = Window.partitionBy("country").orderBy("category")

# Applying window function to calculate average salary within each gender partition
result_df = joined_pin_geo.withColumn("category_count", count("category").over(window_spec))

# Selecting only the relevant columns
df_category_by_country = result_df.select("country", "category", "category_count").distinct()

display(df_category_by_country)

country,category,category_count
Afghanistan,art,2
Afghanistan,christmas,4
Afghanistan,diy-and-crafts,7
Afghanistan,education,19
Afghanistan,event-planning,21
Afghanistan,finance,23
Afghanistan,home-decor,25
Afghanistan,mens-fashion,30
Afghanistan,quotes,40
Afghanistan,tattoos,42


## 2 - Find which was the most popular category each year.

Find how many posts each category had between 2018 and 2022.


Your query should return a DataFrame that contains the following columns:

**post_year**, a new column that contains only the year from the **timestamp** column\
**category**\
**category_count**, a new column containing the desired query output

In [ ]:
# Create a Years column
joined_pin_geo = df_pin.join(df_geo, df_pin["ind"] == df_geo["ind"], how="inner")
joined_pin_geo = joined_pin_geo.withColumn("post_year", year("timestamp"))

# Creating a Window specification
window_spec = Window.partitionBy("post_year").orderBy("category")

# Applying window function to calculate average salary within each gender partition
result_df = joined_pin_geo.withColumn("category_count", count("category").over(window_spec))

# Selecting only the relevant columns
df_category_by_year = result_df.select("post_year", "category", "category_count").distinct()

# Filter years between 2018 and 2022
df_category_by_year_2018_2022 = df_category_by_year.filter(df_category_by_year.post_year >= 2018).filter(df_category_by_year.post_year <= 2022)

display(df_category_by_year_2018_2022)

post_year,category,category_count
2018,art,21
2018,beauty,41
2018,christmas,69
2018,diy-and-crafts,106
2018,education,128
2018,event-planning,142
2018,finance,161
2018,home-decor,176
2018,mens-fashion,195
2018,quotes,222


## 3 - Find the user with most followers in each country. 
The Question is worded strangely, but the user details we can share is their alias which is the "poster_name".

In [ ]:
joined_pin_geo = df_pin.join(df_geo, df_pin["ind"] == df_geo["ind"], how="inner")
joined_pin_geo = joined_pin_geo.select( "follower_count",
                                        "poster_name",
                                        "country"
                                            )


joined_pin_geo.printSchema()
display(joined_pin_geo)

root
-- follower_count: integer (nullable = true)
-- poster_name: string (nullable = true)
-- country: string (nullable = true)

follower_count,poster_name,country
4,Manosbojana Katsareas,Uganda
136000,PUFIK Interiors & Inspirations,Netherlands
394,Sarah Martin,Guyana
5000,Wear24-7,Aruba
784,paupoo,Anguilla
46000,"Life on Summerhill | Home, Holiday Decor & DIY Website",Bermuda
5000,Wear24-7,Aruba
4000000,Etsy,Bouvet Island (Bouvetoya)
137,"Your Home, Reimagined LLC",Australia
137,"Your Home, Reimagined LLC",Australia


In [ ]:
grouped_df = joined_pin_geo .groupBy("country")\
                            .agg(max("follower_count").alias("follower_count"))
display(grouped_df)

country,follower_count
Heard Island and McDonald Islands,487000
French Southern Territories,119000
United States Virgin Islands,4000000
British Indian Ocean Territory (Chagos Archipelago),248000
Holy See (Vatican City State),831
South Georgia and the South Sandwich Islands,75000
Falkland Islands (Malvinas),59000
Saint Vincent and the Grenadines,121
Antarctica (the territory South of 60 deg S),1000000
Bouvet Island (Bouvetoya),5000000


## 4 - Find the most popular category for different age groups


In [ ]:
from pyspark.sql.functions import udf

#Table (base)
joined_pin_user = df_pin    .join(df_user, df_pin["ind"] == df_user["ind"], how="inner")\
                            .select(    "category",
                                        "age"
                            )

#Function (for age groups)
age_range = udf(lambda  age:    "18-24" if (age <= 24 and age >=18 ) else
                                "25-35" if (age <= 35 and age >=25 ) else
                                "36-50" if (age <= 50 and age >=36 ) else
                                "+50"   if (age > 50 ) else '')

#Table (create "age_group" column & drop "age" column)
joined_pin_user = joined_pin_user        .withColumn('age_group', age_range(joined_pin_user.age))\
                                .drop("age")


# Creating a Window specification
window_spec = Window.partitionBy("age_group").orderBy("category")

# Applying window function to calculate average salary within each gender partition
result_df = joined_pin_user.withColumn("category_count", count("category").over(window_spec))

# Selecting only the relevant columns
df_category_by_age_groups = result_df.select("age_group", "category", "category_count").distinct()

df_category_by_age_groups = df_category_by_age_groups   .select("age_group", "category", "category_count")\
                                                        .distinct()\
                                                        .groupBy("age_group", "category")\
                                                        .agg(max("category_count").alias("category_count"))\
                                                        .orderBy(["age_group", "category_count"], ascending=[0,0] )

from pyspark.sql.window import Window
from pyspark.sql.functions import rank, col

window = Window.partitionBy(df_category_by_age_groups['age_group']).orderBy(df_category_by_age_groups['category_count'].desc())

df_category_by_age_groups = df_category_by_age_groups  .select('*', rank().over(window).alias('rank')) \
                                                       .filter(col('rank') <= 2)


display(df_category_by_age_groups)
df_category_by_age_groups.printSchema

age_group,category,category_count,rank
+50,vehicles,98,1
+50,travel,89,2
18-24,vehicles,679,1
18-24,travel,652,2
25-35,vehicles,416,1
25-35,travel,379,2
36-50,vehicles,282,1
36-50,travel,262,2


Out[119]: <bound method DataFrame.printSchema of DataFrame[age_group: string, category: string, category_count: bigint, rank: int]>

## 5 - Find the median follower count for different age groups
What is the median follower count for users in the following age groups:

18-24\
25-35\
36-50\
+50

Your query should return a DataFrame that contains the following columns:

age_group, a new column based on the original age column\
median_follower_count, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import udf

#Table (base)
joined_pin_user = df_pin    .join(df_user, df_pin["ind"] == df_user["ind"], how="inner")\
                            .select(    "follower_count",
                                        "age"
                            )

#Function (for age groups)
age_range = udf(lambda  age:    "18-24" if (age <= 24 and age >=18 ) else
                                "25-35" if (age <= 35 and age >=25 ) else
                                "36-50" if (age <= 50 and age >=36 ) else
                                "50+"   if (age > 50 ) else '')

#Table (create "age_group" column & drop "age" column)
joined_pin_user = joined_pin_user       .withColumn('age_group', age_range(joined_pin_user.age))\
                                        .drop("age")

df_followers_by_age = joined_pin_user.groupBy("age_group").agg(percentile_approx("follower_count", 0.5).alias("median_follower_count")).orderBy("age_group")

display(df_followers_by_age)

age_group,median_follower_count
18-24,104000
25-35,25000
36-50,6000
50+,777


## 6 - Find how many users have joined each year.
Find how many users have joined between 2015 and 2020.


Your query should return a DataFrame that contains the following columns:

post_year, a new column that contains only the year from the timestamp column\
number_users_joined, a new column containing the desired query output

In [ ]:
df_user_join_qty = df_user.withColumn("post_year", year("date_joined"))

df_user_join_qty = df_user_join_qty     .withColumn("post_year", year("date_joined"))\
                                        .filter(df_user_join_qty.post_year >= 2015)\
                                        .filter(df_user_join_qty.post_year <= 2020)\
                                        .drop("ind", "date_joined")

df_user_join_qty = df_user_join_qty.groupBy("post_year").agg({"post_year": "count"})

display(df_user_join_qty)


post_year,count(post_year)
2015,506
2016,559
2017,202


## 7 - Find the median follower count of users based on their joining year
Find the median follower count of users have joined between 2015 and 2020.


Your query should return a DataFrame that contains the following columns:

**post_year**, a new column that contains only the year from the timestamp column\
**median_follower_count**, a new column containing the desired query output

In [ ]:
#Table (base) - Joining user and pin dataframes
joined_pin_user = df_pin    .join(df_user, df_pin["ind"] == df_user["ind"], how="inner")\
                            .select(    "follower_count",
                                        "date_joined"
                            )


#Column (create post_year column)
df_pin_user_years = joined_pin_user     .withColumn("post_year", year("date_joined"))\
                                        .drop("date_joined")


#Group data by joining years and find the median of the follower count
df_pin_user_years_followers = df_pin_user_years     .groupBy("post_year")\
                                                    .agg(percentile_approx("follower_count", 0.5).alias("median_follower_count"))\
                                                    .filter(df_pin_user_years.post_year >= 2015)\
                                                    .filter(df_pin_user_years.post_year <= 2020)\
                                                    .orderBy("post_year")

display(df_pin_user_years_followers)


post_year,median_follower_count
2015,110000
2016,20000
2017,3000


## 8 - Find the median follower count of users based on their joining year and age group
Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.


Your query should return a DataFrame that contains the following columns:

**age_group**, a new column based on the original age column\
**post_year**, a new column that contains only the year from the timestamp column\
**median_follower_count**, a new column containing the desired query output

In [ ]:
#Table (base) - Joining user and pin dataframes
joined_pin_user = df_pin    .join(df_user, df_pin["ind"] == df_user["ind"], how="inner")\
                            .select(    "follower_count",
                                        "date_joined",
                                        "age"
                            )

#Function (for age groups)
age_range = udf(lambda  age:    "18-24" if (age <= 24 and age >=18 ) else
                                "25-35" if (age <= 35 and age >=25 ) else
                                "36-50" if (age <= 50 and age >=36 ) else
                                "50+"   if (age > 50 ) else '')

#Table (create "age_group" column & drop "age" column)
joined_pin_user_age = joined_pin_user   .withColumn('age_group', age_range(joined_pin_user.age))\
                                        .drop("age")
#Table (create post_year column)
df_pin_user_age_years = joined_pin_user_age     .withColumn("post_year", year("date_joined"))\
                                                .drop("date_joined")


# Creating a Window specification
window_spec = Window.partitionBy("post_year").orderBy("age_group")
# Applying window function to calculate average salary within each gender partition
result_df = df_pin_user_age_years.withColumn("median_follower_count", percentile_approx("follower_count", 0.5).over(window_spec))

df_pin_user_age_years_followers = result_df     .select("post_year", "age_group", "median_follower_count").distinct()

display(df_pin_user_age_years_followers)

post_year,age_group,median_follower_count
2015,18-24,221000
2015,25-35,166000
2015,36-50,115000
2015,50+,110000
2016,18-24,41000
2016,25-35,31000
2016,36-50,22000
2016,50+,20000
2017,18-24,5000
2017,25-35,4000
